# Chapter 19: Error Handling & Debugging

Robust error handling separates production-ready applications from experimental projects. Next.js 13+ introduces sophisticated error handling mechanisms through file-based conventions, allowing you to create resilient applications that gracefully recover from failures while providing developers with detailed debugging information.

By the end of this chapter, you'll master Error Boundaries for catching React errors, implement global error handling strategies, leverage the `notFound()` function for missing data scenarios, create custom error experiences, debug Server and Client Components effectively, implement comprehensive logging and monitoring, and recognize common error patterns in Next.js applications.

## 19.1 Error Boundaries

Error Boundaries in the App Router use the `error.tsx` file convention to catch JavaScript errors anywhere in their child component tree.

### Basic Error Boundary Structure

Create error boundaries using the `error.tsx` convention:

```typescript
// app/dashboard/error.tsx
'use client';

import { useEffect } from 'react';

export default function Error({
  error,
  reset,
}: {
  error: Error & { digest?: string };
  reset: () => void;
}) {
  useEffect(() => {
    // Log error to reporting service
    console.error('Dashboard error:', error);
  }, [error]);

  return (
    <div className="min-h-[400px] flex flex-col items-center justify-center p-6">
      <h2 className="text-2xl font-bold text-red-600 mb-4">
        Something went wrong!
      </h2>
      <p className="text-gray-600 mb-6 text-center max-w-md">
        {error.message || 'An unexpected error occurred in the dashboard.'}
      </p>
      <button
        onClick={reset}
        className="px-4 py-2 bg-blue-500 text-white rounded hover:bg-blue-600 transition"
      >
        Try again
      </button>
    </div>
  );
}
```

### Nested Error Boundaries

Error boundaries bubble up to the nearest parent error boundary:

```typescript
// File structure
app/
├── error.tsx              // Root error boundary (catches all)
├── layout.tsx
├── dashboard/
│   ├── error.tsx          // Dashboard-specific errors
│   ├── layout.tsx
│   ├── page.tsx
│   └── settings/
│       ├── error.tsx      // Settings-specific errors
│       └── page.tsx

// app/dashboard/settings/error.tsx
'use client';

export default function SettingsError({ 
  error, 
  reset 
}: { 
  error: Error; 
  reset: () => void; 
}) {
  return (
    <div className="p-6 border border-red-200 bg-red-50 rounded-lg">
      <h3 className="text-lg font-semibold text-red-800 mb-2">
        Settings Error
      </h3>
      <p className="text-red-600 mb-4">
        Failed to load settings. Your changes may not have been saved.
      </p>
      <div className="flex gap-3">
        <button 
          onClick={reset}
          className="px-3 py-1.5 bg-red-600 text-white rounded text-sm"
        >
          Retry
        </button>
        <button 
          onClick={() => window.location.href = '/dashboard'}
          className="px-3 py-1.5 border border-red-300 text-red-700 rounded text-sm"
        >
          Go to Dashboard
        </button>
      </div>
    </div>
  );
}
```

### Error Boundary with Recovery Logic

Implement sophisticated recovery mechanisms:

```typescript
// app/posts/error.tsx
'use client';

import { useEffect, useState } from 'react';
import { useRouter } from 'next/navigation';

export default function PostError({ 
  error, 
  reset 
}: { 
  error: Error & { digest?: string; statusCode?: number }; 
  reset: () => void; 
}) {
  const router = useRouter();
  const [attempts, setAttempts] = useState(0);
  const [isReporting, setIsReporting] = useState(false);

  useEffect(() => {
    // Auto-report to error tracking service
    fetch('/api/error-report', {
      method: 'POST',
      body: JSON.stringify({
        error: error.message,
        stack: error.stack,
        digest: error.digest,
        path: window.location.pathname,
        timestamp: new Date().toISOString(),
      }),
    });
  }, [error]);

  const handleReset = () => {
    setAttempts(prev => prev + 1);
    reset();
  };

  const handleRefresh = () => {
    router.refresh();
  };

  // Give up after 3 attempts
  if (attempts >= 3) {
    return (
      <div className="p-8 text-center">
        <h2 className="text-xl font-bold text-gray-800 mb-4">
          Unable to load content
        </h2>
        <p className="text-gray-600 mb-6">
          We're having trouble loading this page. Please try again later.
        </p>
        <button
          onClick={() => router.push('/')}
          className="px-4 py-2 bg-gray-800 text-white rounded"
        >
          Go Home
        </button>
      </div>
    );
  }

  return (
    <div className="p-6 bg-white border border-red-200 rounded-lg shadow-sm">
      <div className="flex items-start gap-4">
        <div className="text-red-500 text-3xl">⚠️</div>
        <div className="flex-1">
          <h3 className="font-semibold text-gray-900 mb-1">
            Failed to load posts
          </h3>
          <p className="text-sm text-gray-600 mb-4">
            Attempt {attempts + 1} of 3
          </p>
          
          {process.env.NODE_ENV === 'development' && (
            <pre className="bg-gray-100 p-3 rounded text-xs overflow-auto mb-4">
              {error.stack}
            </pre>
          )}
          
          <div className="flex gap-3">
            <button
              onClick={handleReset}
              className="px-4 py-2 bg-blue-500 text-white rounded hover:bg-blue-600"
            >
              Try Again
            </button>
            <button
              onClick={handleRefresh}
              className="px-4 py-2 border border-gray-300 rounded hover:bg-gray-50"
            >
              Refresh Page
            </button>
          </div>
        </div>
      </div>
    </div>
  );
}
```

## 19.2 Global Error Handling

Handle uncaught errors at the root level using `global-error.tsx`.

### Root Error Boundary

Catch errors that bubble up past all other boundaries:

```typescript
// app/global-error.tsx
'use client';

import { useEffect } from 'react';
import * as Sentry from '@sentry/nextjs';

export default function GlobalError({
  error,
  reset,
}: {
  error: Error & { digest?: string };
  reset: () => void;
}) {
  useEffect(() => {
    // Log to Sentry or other error tracking
    Sentry.captureException(error);
  }, [error]);

  return (
    <html>
      <body className="bg-gray-50">
        <div className="min-h-screen flex items-center justify-center p-4">
          <div className="max-w-md w-full bg-white rounded-lg shadow-lg p-8 text-center">
            <div className="w-16 h-16 bg-red-100 rounded-full flex items-center justify-center mx-auto mb-4">
              <span className="text-2xl">😵</span>
            </div>
            
            <h1 className="text-2xl font-bold text-gray-900 mb-2">
              Application Error
            </h1>
            
            <p className="text-gray-600 mb-6">
              A critical error occurred. Our team has been notified and is working on a fix.
            </p>
            
            {process.env.NODE_ENV === 'development' && (
              <div className="bg-red-50 border border-red-200 rounded p-4 mb-6 text-left">
                <p className="text-sm font-mono text-red-800 break-all">
                  {error.message}
                </p>
                {error.digest && (
                  <p className="text-xs text-red-600 mt-2">
                    Error ID: {error.digest}
                  </p>
                )}
              </div>
            )}
            
            <button
              onClick={reset}
              className="w-full px-4 py-2 bg-blue-600 text-white rounded-lg hover:bg-blue-700 transition"
            >
              Try Again
            </button>
          </div>
        </div>
      </body>
    </html>
  );
}
```

### Layout-Level Error Handling

Handle errors in layouts while preserving UI structure:

```typescript
// app/dashboard/layout.tsx
export default function DashboardLayout({
  children,
  stats,
  sidebar,
}: {
  children: React.ReactNode;
  stats: React.ReactNode;
  sidebar: React.ReactNode;
}) {
  return (
    <div className="flex h-screen">
      <aside className="w-64 bg-gray-900 text-white">
        {sidebar}
      </aside>
      <main className="flex-1 overflow-auto bg-gray-50 p-6">
        {children}
      </main>
      <div className="w-80 border-l bg-white">
        {stats}
      </div>
    </div>
  );
}

// app/dashboard/@stats/error.tsx
'use client';

export default function StatsError() {
  return (
    <div className="p-4 text-sm text-gray-500">
      <p>Statistics temporarily unavailable</p>
    </div>
  );
}

// app/dashboard/@sidebar/error.tsx  
'use client';

export default function SidebarError() {
  return (
    <div className="p-4 text-white">
      <p>Menu loading failed</p>
    </div>
  );
}
```

## 19.3 notFound() Function

Handle missing resources gracefully using the `notFound()` function and `not-found.tsx` files.

### Using notFound() for Missing Data

Invoke `notFound()` when data doesn't exist:

```typescript
// app/blog/[slug]/page.tsx
import { notFound } from 'next/navigation';
import { db } from '@/lib/db';

export default async function BlogPost({ 
  params 
}: { 
  params: { slug: string } 
}) {
  const post = await db.post.findUnique({
    where: { slug: params.slug },
  });

  if (!post) {
    notFound(); // Renders nearest not-found.tsx
  }

  if (!post.published && !isAdmin()) {
    notFound(); // Hide unpublished posts from non-admins
  }

  return (
    <article>
      <h1>{post.title}</h1>
      <div>{post.content}</div>
    </article>
  );
}

// app/blog/[slug]/not-found.tsx
import Link from 'next/link';

export default function PostNotFound() {
  return (
    <div className="min-h-[400px] flex flex-col items-center justify-center text-center p-6">
      <h2 className="text-4xl font-bold text-gray-300 mb-4">404</h2>
      <h3 className="text-xl font-semibold text-gray-900 mb-2">
        Post Not Found
      </h3>
      <p className="text-gray-600 mb-6 max-w-md">
        The blog post you're looking for doesn't exist or has been removed.
      </p>
      <div className="flex gap-4">
        <Link 
          href="/blog"
          className="px-4 py-2 bg-blue-500 text-white rounded hover:bg-blue-600"
        >
          Browse All Posts
        </Link>
        <Link 
          href="/"
          className="px-4 py-2 border border-gray-300 rounded hover:bg-gray-50"
        >
          Go Home
        </Link>
      </div>
    </div>
  );
}
```

### Dynamic Route Segment notFound()

Handle missing data in nested segments:

```typescript
// app/shop/[category]/[product]/page.tsx
import { notFound } from 'next/navigation';

export default async function ProductPage({
  params,
}: {
  params: { category: string; product: string };
}) {
  // Validate category exists
  const category = await getCategory(params.category);
  if (!category) {
    notFound();
  }

  // Validate product exists in category
  const product = await getProduct(params.product, category.id);
  if (!product) {
    notFound();
  }

  return <ProductDetails product={product} />;
}

// app/shop/[category]/[product]/not-found.tsx
export default function ProductNotFound() {
  return (
    <div className="container mx-auto py-12 text-center">
      <h1 className="text-2xl font-bold mb-4">Product Not Available</h1>
      <p className="text-gray-600">
        This product may have been discontinued or moved.
      </p>
    </div>
  );
}

// app/shop/[category]/not-found.tsx  
export default function CategoryNotFound() {
  return (
    <div className="container mx-auto py-12 text-center">
      <h1 className="text-2xl font-bold mb-4">Category Not Found</h1>
      <p className="text-gray-600">
        We couldn't find this product category.
      </p>
    </div>
  );
}
```

### Parallel Route notFound()

Handle not-found states in parallel routes:

```typescript
// app/@modal/(.)photo/[id]/page.tsx
import { notFound } from 'next/navigation';

export default async function PhotoModal({ 
  params 
}: { 
  params: { id: string } 
}) {
  const photo = await getPhoto(params.id);
  
  if (!photo) {
    notFound(); // Will render app/@modal/not-found.tsx
  }
  
  return <PhotoDialog photo={photo} />;
}

// app/@modal/not-found.tsx
'use client';

import { useRouter } from 'next/navigation';

export default function ModalNotFound() {
  const router = useRouter();
  
  // Close modal if content not found
  useEffect(() => {
    router.back();
  }, [router]);
  
  return null;
}
```

## 19.4 Custom Error Pages

Create branded error experiences for different HTTP status codes.

### HTTP Status Error Handling

Handle specific error types differently:

```typescript
// lib/errors.ts
export class APIError extends Error {
  constructor(
    message: string,
    public statusCode: number,
    public code: string
  ) {
    super(message);
  }
}

export class ValidationError extends APIError {
  constructor(message: string, public fields: Record<string, string>) {
    super(message, 400, 'VALIDATION_ERROR');
  }
}

export class AuthenticationError extends APIError {
  constructor(message: string = 'Unauthorized') {
    super(message, 401, 'AUTHENTICATION_ERROR');
  }
}

export class AuthorizationError extends APIError {
  constructor(message: string = 'Forbidden') {
    super(message, 403, 'AUTHORIZATION_ERROR');
  }
}

// app/error.tsx
'use client';

import { APIError } from '@/lib/errors';

export default function ErrorPage({ 
  error, 
  reset 
}: { 
  error: Error | APIError; 
  reset: () => void; 
}) {
  let statusCode = 500;
  let title = 'Internal Server Error';
  let description = 'Something went wrong on our end.';

  if (error instanceof APIError) {
    statusCode = error.statusCode;
    
    switch (error.statusCode) {
      case 400:
        title = 'Bad Request';
        description = 'The request was invalid.';
        break;
      case 401:
        title = 'Unauthorized';
        description = 'Please log in to access this resource.';
        break;
      case 403:
        title = 'Forbidden';
        description = "You don't have permission to access this resource.";
        break;
      case 404:
        title = 'Not Found';
        description = "The requested resource doesn't exist.";
        break;
      case 429:
        title = 'Too Many Requests';
        description = 'Please slow down and try again later.';
        break;
    }
  }

  return (
    <div className="min-h-screen flex items-center justify-center bg-gray-50">
      <div className="text-center">
        <h1 className="text-6xl font-bold text-gray-900 mb-4">{statusCode}</h1>
        <h2 className="text-2xl font-semibold text-gray-700 mb-2">{title}</h2>
        <p className="text-gray-600 mb-8">{description}</p>
        <button 
          onClick={reset}
          className="px-6 py-3 bg-blue-600 text-white rounded-lg hover:bg-blue-700"
        >
          Try Again
        </button>
      </div>
    </div>
  );
}
```

### 404 Page Customization

Create engaging not-found experiences:

```typescript
// app/not-found.tsx
import Link from 'next/link';
import { Search } from '@/components/search';

export default function NotFound() {
  return (
    <div className="min-h-screen bg-gradient-to-b from-gray-50 to-white flex items-center justify-center p-4">
      <div className="max-w-2xl w-full text-center">
        <div className="mb-8">
          <div className="inline-block p-6 rounded-full bg-blue-50 mb-4">
            <svg className="w-16 h-16 text-blue-500" fill="none" viewBox="0 0 24 24" stroke="currentColor">
              <path strokeLinecap="round" strokeLinejoin="round" strokeWidth={2} d="M9.172 16.172a4 4 0 015.656 0M9 10h.01M15 10h.01M21 12a9 9 0 11-18 0 9 9 0 0118 0z" />
            </svg>
          </div>
          <h1 className="text-5xl font-bold text-gray-900 mb-4">404</h1>
          <h2 className="text-2xl font-semibold text-gray-700 mb-4">
            Page Not Found
          </h2>
          <p className="text-gray-600 mb-8 max-w-md mx-auto">
            Sorry, we couldn't find the page you're looking for. It might have been moved or deleted.
          </p>
        </div>

        <div className="bg-white p-6 rounded-xl shadow-sm border border-gray-200 mb-8">
          <p className="text-sm text-gray-600 mb-4">Looking for something specific?</p>
          <Search />
        </div>

        <div className="flex flex-wrap justify-center gap-4">
          <Link 
            href="/"
            className="px-6 py-3 bg-blue-600 text-white rounded-lg hover:bg-blue-700 transition"
          >
            Go Home
          </Link>
          <Link 
            href="/sitemap"
            className="px-6 py-3 border border-gray-300 rounded-lg hover:bg-gray-50 transition"
          >
            View Sitemap
          </Link>
          <button 
            onClick={() => window.history.back()}
            className="px-6 py-3 border border-gray-300 rounded-lg hover:bg-gray-50 transition"
          >
            Go Back
          </button>
        </div>
      </div>
    </div>
  );
}
```

## 19.5 Debugging Techniques

Effective debugging strategies for both Server and Client Components.

### Server Component Debugging

Debug Server Components during development:

```typescript
// app/debug/page.tsx
import { cookies, headers } from 'next/headers';

export default async function DebugPage() {
  // Log server-side data (appears in terminal, not browser)
  console.log('Rendering Debug Page');
  
  const cookieStore = cookies();
  const headersList = headers();
  
  // Inspect request data
  const debugInfo = {
    timestamp: new Date().toISOString(),
    userAgent: headersList.get('user-agent'),
    acceptLanguage: headersList.get('accept-language'),
    cookies: Array.from(cookieStore.getAll()).map(c => c.name),
    env: {
      nodeEnv: process.env.NODE_ENV,
      hasDatabaseUrl: !!process.env.DATABASE_URL,
    },
  };
  
  // Force error to test error boundary
  if (process.env.FORCE_ERROR === 'true') {
    throw new Error('Forced debug error');
  }

  return (
    <div className="p-8 font-mono text-sm">
      <h1 className="text-2xl font-bold mb-4">Debug Information</h1>
      <pre className="bg-gray-100 p-4 rounded overflow-auto">
        {JSON.stringify(debugInfo, null, 2)}
      </pre>
    </div>
  );
}
```

### Source Maps and Stack Traces

Enable better debugging in production:

```javascript
// next.config.js
module.exports = {
  // Enable source maps in production for better error tracking
  productionBrowserSourceMaps: true,
  
  // Configure logging
  logging: {
    fetches: {
      fullUrl: true,
    },
  },
  
  // Experimental: Enhanced debugging
  experimental: {
    // Server Actions debugging
    serverActions: {
      bodySizeLimit: '2mb',
    },
  },
};
```

### React DevTools Integration

Use React DevTools with Server Components:

```typescript
// components/debug-provider.tsx
'use client';

import { useEffect } from 'react';

export function DebugProvider({ children }: { children: React.ReactNode }) {
  useEffect(() => {
    // Enable React DevTools highlighting
    if (process.env.NODE_ENV === 'development') {
      // @ts-ignore
      window.__REACT_DEVTOOLS_GLOBAL_HOOK__?.onCommitFiberRoot?.();
    }
  }, []);

  return <>{children}</>;
}

// Debug boundaries to identify which component threw
export function DebugBoundary({ 
  name, 
  children 
}: { 
  name: string; 
  children: React.ReactNode; 
}) {
  console.log(`[Debug] Rendering: ${name}`);
  
  return (
    <div data-debug-name={name}>
      {children}
    </div>
  );
}
```

### Server Action Debugging

Debug Server Actions with detailed logging:

```typescript
// app/actions.ts
'use server';

import { revalidatePath } from 'next/cache';

export async function debugAction(formData: FormData) {
  console.log('Server Action invoked:', new Date().toISOString());
  
  try {
    // Log incoming data
    console.log('Form data:', Object.fromEntries(formData));
    
    // Simulate processing
    await new Promise(resolve => setTimeout(resolve, 1000));
    
    // Log success
    console.log('Action completed successfully');
    
    revalidatePath('/');
    return { success: true };
    
  } catch (error) {
    // Detailed error logging
    console.error('Action failed:', {
      error: error instanceof Error ? error.message : 'Unknown',
      stack: error instanceof Error ? error.stack : undefined,
      formData: Object.fromEntries(formData),
    });
    
    throw error; // Re-throw to trigger error boundary
  }
}
```

## 19.6 Logging and Monitoring

Implement structured logging and external monitoring for production applications.

### Structured Logging

Create a consistent logging strategy:

```typescript
// lib/logger.ts
type LogLevel = 'debug' | 'info' | 'warn' | 'error';

interface LogEntry {
  level: LogLevel;
  message: string;
  timestamp: string;
  context?: Record<string, any>;
  error?: Error;
}

class Logger {
  private static instance: Logger;
  
  static getInstance(): Logger {
    if (!Logger.instance) {
      Logger.instance = new Logger();
    }
    return Logger.instance;
  }

  private log(level: LogLevel, message: string, context?: Record<string, any>, error?: Error) {
    const entry: LogEntry = {
      level,
      message,
      timestamp: new Date().toISOString(),
      context,
      error,
    };

    // Console output for development
    if (process.env.NODE_ENV === 'development') {
      console[level](`[${entry.timestamp}] ${level.toUpperCase()}: ${message}`, context || '');
      if (error) console.error(error);
    }

    // Production: Send to logging service
    if (process.env.NODE_ENV === 'production') {
      this.sendToLogService(entry);
    }
  }

  private async sendToLogService(entry: LogEntry) {
    // Send to Datadog, LogRocket, or custom endpoint
    try {
      await fetch(process.env.LOG_ENDPOINT || '', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify(entry),
      });
    } catch {
      // Silent fail for logging
    }
  }

  debug(message: string, context?: Record<string, any>) {
    this.log('debug', message, context);
  }

  info(message: string, context?: Record<string, any>) {
    this.log('info', message, context);
  }

  warn(message: string, context?: Record<string, any>) {
    this.log('warn', message, context);
  }

  error(message: string, error: Error, context?: Record<string, any>) {
    this.log('error', message, context, error);
  }
}

export const logger = Logger.getInstance();
```

### Error Tracking Integration

Integrate Sentry or similar services:

```typescript
// lib/sentry.ts
import * as Sentry from '@sentry/nextjs';

export function initSentry() {
  Sentry.init({
    dsn: process.env.NEXT_PUBLIC_SENTRY_DSN,
    environment: process.env.NODE_ENV,
    tracesSampleRate: 1.0,
    
    beforeSend(event) {
      // Filter out specific errors
      if (event.exception?.values?.[0]?.value?.includes('ResizeObserver')) {
        return null;
      }
      return event;
    },
  });
}

// Usage in error boundaries
// app/error.tsx
'use client';

import * as Sentry from '@sentry/nextjs';
import { useEffect } from 'react';

export default function ErrorBoundary({ 
  error, 
  reset 
}: { 
  error: Error; 
  reset: () => void; 
}) {
  useEffect(() => {
    Sentry.captureException(error, {
      tags: { section: 'dashboard' },
      extra: { 
        path: window.location.pathname,
        userAgent: navigator.userAgent,
      },
    });
  }, [error]);

  return <div>Error occurred</div>;
}
```

### Performance Monitoring

Track Core Web Vitals and performance metrics:

```typescript
// app/web-vitals.tsx
'use client';

import { useReportWebVitals } from 'next/web-vitals';

export function WebVitals() {
  useReportWebVitals((metric) => {
    // Send to analytics
    const body = JSON.stringify(metric);
    const url = '/api/analytics/web-vitals';

    // Use `navigator.sendBeacon()` if available
    if (navigator.sendBeacon) {
      navigator.sendBeacon(url, body);
    } else {
      fetch(url, { body, method: 'POST', keepalive: true });
    }
    
    // Log in development
    console.log(metric);
  });

  return null;
}

// app/layout.tsx
import { WebVitals } from './web-vitals';

export default function RootLayout({ children }: { children: React.ReactNode }) {
  return (
    <html>
      <body>
        <WebVitals />
        {children}
      </body>
    </html>
  );
}
```

## 19.7 Common Error Patterns

Recognize and resolve frequently encountered errors in Next.js applications.

### Hydration Mismatch Errors

Fix hydration mismatches between server and client:

```typescript
// Problem: Dates or random values differ between server/client
// app/page.tsx (Bad)
export default function Page() {
  return <div>{new Date().toLocaleString()}</div>; // Hydration error!
}

// Solution 1: Use client component for dynamic content
'use client';
export default function Page() {
  const [date, setDate] = useState('');
  
  useEffect(() => {
    setDate(new Date().toLocaleString());
  }, []);
  
  return <div>{date}</div>;
}

// Solution 2: Use suppressHydrationWarning for unavoidable mismatches
export default function Page() {
  return (
    <div suppressHydrationWarning>
      {new Date().toLocaleString()}
    </div>
  );
}

// Solution 3: Consistent initial render
export default function Page() {
  const [mounted, setMounted] = useState(false);
  
  useEffect(() => setMounted(true), []);
  
  return (
    <div>
      {mounted ? new Date().toLocaleString() : 'Loading...'}
    </div>
  );
}
```

### Server Component Serialization Errors

Handle serialization limits:

```typescript
// Problem: Passing non-serializable data to Client Components
// app/page.tsx (Bad)
export default async function Page() {
  const data = await fetchData();
  
  return <ClientComponent data={data} onClick={() => {}} />; 
  // Error: Functions can't be passed to Client Components
}

// Solution: Pass only serializable data
export default async function Page() {
  const data = await fetchData();
  
  return (
    <ClientComponent 
      data={data} 
      // Pass action as string or use Server Action
    />
  );
}

// app/actions.ts
'use server';
export async function handleClick() {
  // Server-side logic
}

// Client Component
'use client';
export function ClientComponent({ data }: { data: any }) {
  return <button onClick={() => handleClick()}>Click</button>;
}
```

### Dynamic Server Usage Errors

Fix dynamic API usage errors:

```typescript
// Problem: Using dynamic APIs without configuring dynamic behavior
// app/page.tsx (Bad)
export default async function Page() {
  const cookie = cookies().get('session'); // Error!
  return <div>{cookie?.value}</div>;
}

// Solution 1: Force dynamic rendering
export const dynamic = 'force-dynamic';

export default async function Page() {
  const cookie = cookies().get('session');
  return <div>{cookie?.value}</div>;
}

// Solution 2: Use revalidate = 0 for SSR
export const revalidate = 0;

// Solution 3: Handle static generation fallback
export default async function Page() {
  let cookie;
  
  try {
    cookie = cookies().get('session');
  } catch {
    // During static generation, cookies() throws
    cookie = null;
  }
  
  return <div>{cookie?.value || 'No session'}</div>;
}
```

### Fetch Cache Errors

Resolve fetch caching issues:

```typescript
// Problem: Stale data or unexpected caching
// app/page.tsx
export default async function Page() {
  const data = await fetch('/api/data'); // Might be cached unexpectedly
  return <div>{data}</div>;
}

// Solution 1: Explicit cache control
export default async function Page() {
  const data = await fetch('/api/data', {
    cache: 'no-store', // Always fresh
  });
  return <div>{data}</div>;
}

// Solution 2: Revalidation
export default async function Page() {
  const data = await fetch('/api/data', {
    next: { revalidate: 60 }, // ISR
  });
  return <div>{data}</div>;
}

// Solution 3: Tag-based revalidation
export default async function Page() {
  const data = await fetch('/api/data', {
    next: { tags: ['dashboard-data'] },
  });
  return <div>{data}</div>;
}
```

## Key Takeaways from Chapter 19

1. **Error Boundaries**: Create `error.tsx` files to catch React errors in nested routes. Error boundaries must be Client Components (`'use client'`) and receive `error` and `reset` props for displaying error details and retrying failed renders. Errors bubble up to the nearest parent error boundary.

2. **Global Error Handling**: Use `global-error.tsx` at the app root to catch uncaught errors that escape all other boundaries. This requires its own `<html>` and `<body>` tags since it replaces the entire layout. Always include error tracking service integration (Sentry, LogRocket) here.

3. **notFound() Function**: Call `notFound()` (imported from `next/navigation`) when data doesn't exist to render the nearest `not-found.tsx` file. This is distinct from error boundaries—use it for expected "missing resource" scenarios (404s) rather than application errors (500s).

4. **Custom Error Pages**: Create branded 404 experiences in root or nested `not-found.tsx` files, and handle specific HTTP status codes by creating custom error classes (APIError) that error boundaries can interpret to show appropriate messages and recovery options.

5. **Debugging Techniques**: Debug Server Components using terminal console logs (not browser console), enable `productionBrowserSourceMaps` for production debugging, use `logging.fetches` in next.config.js to trace data fetching, and leverage React DevTools with Server Components by inspecting the component tree structure.

6. **Logging and Monitoring**: Implement structured logging with severity levels (debug, info, warn, error) that sends to external services in production while remaining readable in development. Use `useReportWebVitals` to track Core Web Vitals, and integrate error tracking services (Sentry) in both error boundaries and global error handlers.

7. **Common Error Patterns**: Fix hydration mismatches using `useEffect` for client-only data, `suppressHydrationWarning` for unavoidable differences, or `mounted` state checks. Avoid passing non-serializable data (functions, dates, class instances) from Server to Client Components. Handle "Dynamic server usage" errors by adding `export const dynamic = 'force-dynamic'` or `revalidate = 0` when using dynamic APIs like `cookies()` or `headers()`.

## Coming Up Next

**Chapter 20: Testing Strategies**

Now that you can handle errors gracefully, it's time to ensure your code works correctly before it reaches production. In Chapter 20, we'll explore Unit Testing with Jest and Vitest, Component Testing with React Testing Library, End-to-End Testing with Playwright, Integration Testing patterns specific to Next.js, testing Server Components, mocking strategies, and Test-Driven Development workflows. You'll learn how to build comprehensive test suites that give you confidence in your application's reliability.